# Домашнее задание 2 - Локализация ключевых точек
<br>

Задание выполнил(а): Алеся Демешко
<br>

**Дедлайн:** 17.11.2020 23:59


### О задании
В этом задании вы познакомитесь с функционалом SAS для реализации сетей локализации ключевых точек, а так же построите и обучите такую сеть самостоятельно.
<br>

**Задание построено следующим образом:**<br>
1) Задание 1: Загрузка данных в CAS - 1 балл<br>
2) Задание 2: Соединение таблиц - 3 балла<br>
3) Задание 3: Подготовка данных для обучения - 2 балла<br>
4) Задание 4: Построение модели - 4 балла<br>
5) Задание 5: Обучение модели - 2 балла<br>
6) Задание 6: Проверка и визуализация результатов работы сети - 3 балла<br>
7) Задание 7 (дополнительное) - до +2 баллов

### Формат сдачи

После того, как вы выполните домашную работу, поместите jupyter notebook с выполнеными заданиями, а так же все SAS-таблицы с сохраненными моделями в закрытый git-репозиторий на GitHub. Предоставьте доступ к репозиторию аккаунтам GazizovMarat и Ala1s (добавив в свой реппозиторий в качестве коллабораторов).

### Важно 
Для того, чтобы ваши результаты были воспроизводимыми для проверки, указывайте seed при вызове методов, где это возможно. Напоминания об этом так же содержатся в тексте заданий.

In [47]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm

from swat import *
from dlpy import Model, Sequential
from dlpy.layers import *
from dlpy.images import ImageTable
from dlpy.splitting import three_way_split

**Подключение к серверу CAS**<br>
Подключитесь к серверу CAS под своим логином и паролем.<br>
Загрузите следующие Action Sets: 'image' и 'deepLearn'.<br>

In [48]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = r"/tmp/4fix/trustedcerts.pem"

# Создаем новую сессию CAS:

# Ваш код здесь
s = CAS('tiger.cs.msu.ru', 5570, username='stdusr19', password='wx6BEbDp')

# Загружаем action sets:

# Ваш код здесь
s.loadactionset('image')
s.loadactionset('deepLearn')
s.loadactionset('datastep')

NOTE: Added action set 'image'.
NOTE: Added action set 'deepLearn'.
NOTE: Added action set 'datastep'.


[actionset]

 'datastep'

+ Elapsed: 0.000622s, sys: 0.000553s, mem: 0.221mb

**Набор данных**<br>
Вам предлагается набор данных с фотографиями кошек. Сами фотографии располагаются в папке images. Для каждой фотографии имеется разметка, содержащяя координаты 9 ключевых точек мордочки кошки, точки в разметке представлены через пробел в следующем порядке:
- Number of points (default is 9)
- Left Eye
- Right Eye
- Mouth
- Left Ear-1
- Left Ear-2
- Left Ear-3
- Right Ear-1
- Right Ear-2
- Right Ear-3

<br>
Пример изображения:<br>

![cat](cat1.PNG)

<br>
Первое число - количество точек, оно нам не потребуется.<br>
Разметка для каждого изображения находится в отдельном файле, все эти файлы с разметкой содержатся в папке points. Название каждого такого файла совпадает с названием изображения, но имеют расширение '.cat', но не переживайте - это простые текстовые файлы. <br>
Пример:<br>
Название файла изображения - 00000001_000.jpg <br>
Название файла с разметкой - 00000001_000.jpg.cat

**Задание 1: Загрузка данных в CAS**<br><br>
a) Перед тем, как использовать набор данных, потребуется решить несколько проблем с ним. <br><br>
Во-первых, для того чтобы загрузить разметку в CAS потребуется сформировать из них таблицу. Дополните код ниже, чтобы сделать это. <br><br>
Заметьте, что одна из строчек кода, уже написанного за вас, помогает отсеивать разметку, в которой не все ключевые точки присутствуют на изображении - в таком случае в разметке присутствуют отрицательные числа. Такие примеры нам не потребуются, поэтому код, сохраняющий разметку для изображений, напишите внутри этого блока if.

In [49]:
points_data = []
a_path = "/home/stdusr19/SAS-CV-2020-Homework2/cats/points/*.jpg.cat"
for path in glob(a_path):
    with open(path,'r') as f:
        points = f.read().split(' ')[1:-1]
        points = [int(x) for x in points]
        if any(x<0 for x in points) == False:
            points_data.append(points)

In [50]:
labels = ['_filename_0',
'left_eye_x','left_eye_y',
'right_eye_x','right_eye_y',
'mouth_x','mouth_y',
'left_ear1_x','left_ear1_y',
'left_ear2_x','left_ear2_y',
'left_ear3_x','left_ear3_y',
'right_ear1_x','right_ear1_y',
'right_ear2_x','right_ear2_y',
'right_ear3_x','right_ear3_y']

In [51]:
data = pd.DataFrame.from_records(points_data)

In [52]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,175,160,239,162,199,199,149,121,137,78,166,93,281,101,312,96,296,133
1,318,222,340,224,327,238,301,199,304,182,321,196,341,197,359,183,361,203
2,167,173,242,185,191,247,112,105,118,0,171,74,270,90,353,35,332,147
3,115,122,179,121,133,169,95,70,93,17,131,46,233,39,281,9,224,69
4,375,263,436,260,409,296,332,226,319,157,363,186,439,179,472,146,475,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9000,311,133,364,136,333,186,286,96,267,45,315,62,377,66,417,49,406,105
9001,497,322,563,324,551,367,409,300,393,218,450,254,533,258,587,227,581,306
9002,178,174,213,181,190,204,152,154,160,109,186,141,216,147,253,129,242,173
9003,256,131,288,131,274,158,246,106,235,83,259,97,290,97,318,80,292,106


b) Также, все координаты ключевых точек указаны в абсолютных коорданатах - соотвественно, если мы изменим размер изображения, координаты ключевых точек придётся пересчитывать. Давайте переведем их в относительные координаты - для этого потребуется поделить координаты по x на ширину изображения, а координаты по y на высоту изображения. Обратите внимание, что все изображения различного размера. Также обратите внимание на тип данных столбцов в dataframe - если тип целочисленный, то посчитанные вами значения будут округляться к нулю или единице, что, конечно же, неверно.  

In [60]:
# Ваш код здесь
points_data=[]
names = []
for path in glob(a_path):
    with open(path,'r') as f:
        points = f.read().split(' ')[1:-1]
        points = [int(x) for x in points]
        if any(x<0 for x in points) == False:
            img_path_list = path.split('points')
            img_path_list.insert(1, 'images')
            img_path_list[-1] = img_path_list[-1][:-4:]
            img_path = "".join(img_path_list)
            img = cv2.imread(img_path, 0)
            height, width = img.shape[0], img.shape[1]
            for i in range(len(points)):
                if i%2==0:
                    points[i]=points[i]*1./width
                else:
                    points[i]=points[i]*1./height
            points_data.append(points)
            names.append(img_path_list[-1][1:])

In [61]:
data = pd.DataFrame.from_records(points_data, columns=labels[1:])

In [63]:
data.insert(loc=0, column='_filename_0', value = names)

c) Сохраните набор данных в формате csv, а затем загрузите эту таблицу в CAS с помощью функции upload_file.

In [65]:
# Ваш код здесь
data.to_csv(path_or_buf = 'new_data.csv', index = False)
tbl = s.upload_file("/home/stdusr19/SAS-CV-2020-Homework2/new_data.csv")

NOTE: Cloud Analytic Services made the uploaded file available as table NEW_DATA in caslib CASUSER(stdusr19).
NOTE: The table NEW_DATA has been created in caslib CASUSER(stdusr19) from binary data uploaded to Cloud Analytic Services.


d) Создайте список названий переменных, в которых лежат координаты ключевых точек - он нам понадобится в дальнейшем.

In [66]:
labels = ['_filename_0',
'left_eye_x','left_eye_y',
'right_eye_x','right_eye_y',
'mouth_x','mouth_y',
'left_ear1_x','left_ear1_y',
'left_ear2_x','left_ear2_y',
'left_ear3_x','left_ear3_y',
'right_ear1_x','right_ear1_y',
'right_ear2_x','right_ear2_y',
'right_ear3_x','right_ear3_y']

e) Загрузите изображения из папки images в ImageTable.

In [67]:
# Ваш код здесь
img_path = '/home/stdusr19/SAS-CV-2020-Homework2/cats/images'
img = ImageTable.load_files(conn=s, path=img_path)

**Задание 2: Соединение таблиц**<br><br>
a) Соедините таблицу с изображениями и таблицу с казметкой с помощью action dlJoin. 
b) Затем с помощью конструктора ImageTable.from_table превратите уже соединенную таблицу в ImageTable для дальнейшей работы с ней.

In [81]:
# a)
# Ваш код здесь

s.deeplearn.dljoin(annotatedtable=img, table=tbl, 
                      casout=dict(name='merged_table', replace=True), id='_filename_0')

,casLib,Name,Rows,Columns,casTable
0,CASUSER(stdusr19),merged_table,9005,22,"CASTable('merged_table', caslib='CASUSER(stdus..."


In [82]:
merged_table

CASTable(CASTable({'OutputCasTables':               casLib          Name  Rows  Columns  \
0  CASUSER(stdusr19)  merged_table  9005       22   

                                            casTable  
0  CASTable('merged_table', caslib='CASUSER(stdus...  }))

In [84]:
# b)
# Ваш код здесь
a = s.CASTable('merged_table')
merged_table = ImageTable.from_table(a)

**Задание 3: Подготовка данных для обучения**<br><br>
а) Измените размер изображений на 96x96 с помощью функции resize ImageTable.<br>
b) Перемешайте данные с помощью функции action shuffle.<br>
c) Разделите набор данных на обучающий, валидационный и тестовых с помощью функции three_way_split. Процент данных в валидационном наборе данных - 20%, в тестовом - 20%.<br>

In [85]:
# a)
# Ваш код здесь
merged_table.resize(height=96, width=96, inplace=True)

In [86]:
# b)
# Ваш код здесь
merged_table.shuffle(casout=dict(name='merged_table', replace=True))

[caslib]

 'CASUSER(stdusr19)'

[tableName]

 'MERGED_TABLE'

[casTable]

 CASTable('MERGED_TABLE', caslib='CASUSER(stdusr19)')

+ Elapsed: 0.711s, user: 0.103s, sys: 0.954s, mem: 154mb

In [90]:
# c)
# Ваш код здесь
train_data, val_data, test_data = three_way_split(merged_table, valid_rate=20, test_rate=20)

**Задание 4: Построение модели**<br><br>
Постройте модель CNN, схема которой изображена на картинке. Особенности модели не отраженные на рисунке: в слое Conv предшествующем BatchNormalization отсутствуют bias, а в Dense слое присутствует дропаут с вероятностью 0.2. Полученная вами сеть должна показывать Loss Error на тестовом наборе данных более 0.21.
![model](kpnetwork.PNG)

In [91]:
model = Sequential(s, model_table='cnn_model')
model.add(InputLayer(1,96,96))

# Ваш код здесь
model.add(Conv2d(72, 4, 4))
model.add(Pooling(2, 2))
model.add(Conv2d(48, 2, 2))
model.add(BN())
model.add(Dense(48))

model.add(Keypoints(n=18, name='kp1'))

NOTE: Input layer added.
NOTE: Convolution layer added.
NOTE: Pooling layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Fully-connected layer added.
NOTE: Keypoints layer added.
NOTE: Model compiled successfully.


**Задание 5: Обучение модели**<br><br>
Обучите модель, дополнив код ниже. Формат входных данных в model.fit укажите с помощью параметра data_specs.

In [92]:
from dlpy.model import *
from dlpy.model import Optimizer

#Подберите самостоятельно
lr = 

optimizer = Optimizer(mini_batch_size=12, log_level=2, max_epochs=20, reg_l2=0.0005,
                      algorithm=MomentumSolver(clip_grad_max=1000, clip_grad_min=-1000,
                                               learning_rate=lr, step_size=30))

data_specs = # Ваш код здесь
train_history = model.fit( ,# Ваш код здесь
                          seed=42)

SyntaxError: invalid syntax (<ipython-input-92-60742b8bfbf4>, line 5)

**Задание 6: Проверка и визуализация результатов работы сети**<br><br>
a) Получите предсказания на тестовом наборе данных c помощью функции predict.<br>
b) Дополните код функции plot_landmarks, предназначенной для визуализации предсказаний модели. С помощью вызова этой функции отоборазите результаты из тестового набора данных.

In [ ]:
# a)
# Ваш код здесь

In [ ]:
# b)
def plot_landmarks(conn, data, targets, image_list=range(4), n_col=2, figsize=None):
    import cv2
    n_images = len(image_list)
    num_obs = min(len(data), n_images)
    res_img = conn.fetchimages(table=data, to=num_obs).Images
    res_label = conn.fetch(data, fetchvars=targets, to=num_obs).Fetch

    if n_images > n_col:
        n_row = n_images // n_col + 1
    else:
        n_row = 1
        n_col = n_images

    if figsize is None:
        figsize = (16, 16 // n_col * n_row)

    fig = plt.figure(figsize=figsize)

    idx_img = 1
    for i in image_list:
        # Ваш код здесь
    plt.show()

In [ ]:
plot_landmarks(#Ваш код здесь

**Задание 7 (дополнительное):**<br><br>
Перепишите функцию plot_landmarks так, чтобы различные ключевые точки отображались различными цветами. То есть, например, точка левого глаза всегда была синей, правого - зеленой и т.д.

In [ ]:
#Ваш код здесь